In [3]:
import sys 
sys.path.append('../Python-modules/') #For bee_helpers and file_helpers 
from bee_helpers import calc_trip_lengths, get_forager_bee_ids, get_random_bee_ids, get_all_bee_ids
from file_helpers import delete_detection_caches_for_date, detections_to_presence, create_presence_cache_filename, create_presence_cache_filename, cache_location_prefix, cache_detections_from_database, cache_death_dates, last_days_caches
from datetime import timedelta, datetime
from pathlib import Path
from bee_cache import Cache, CacheType, CacheFormat; c=Cache()
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
c.load('real_forager_lives')

,born,died,lifespan,foraging_min_date,foraging_max_date,foraging_min_age,foraging_max_age
bee_id,,,,,,,
135,2016-07-19,2016-08-24,36.0,2016-08-01,2016-08-05,13 days,17 days
199,2016-07-19,2016-08-20,32.0,2016-07-31,2016-08-18,12 days,30 days
217,2016-07-19,2016-08-03,15.0,2016-07-28,2016-08-02,9 days,14 days
220,2016-07-19,2016-08-06,18.0,2016-08-01,2016-08-05,13 days,17 days
228,2016-07-19,2016-08-10,22.0,2016-07-31,2016-08-05,12 days,17 days
233,2016-07-19,2016-08-15,27.0,2016-07-28,2016-08-05,9 days,17 days
253,2016-07-19,2016-08-05,17.0,2016-07-31,2016-08-03,12 days,15 days
319,2016-07-19,2016-08-23,35.0,2016-07-28,2016-08-22,9 days,34 days
337,2016-07-19,2016-08-18,30.0,2016-08-08,2016-08-18,20 days,30 days


In [13]:
c.load('foragers_from_groups')

,bee_id,group_id,date,location
bee_id,,,,
769,769,1,2016-07-28,Stock-F1
818,818,1,2016-07-28,Stock-F1
319,319,1,2016-07-28,Stock-F1
644,644,1,2016-07-28,Stock-F1
392,392,1,2016-07-28,Stock-F1
648,648,1,2016-07-28,Stock-F1
400,400,1,2016-07-28,Stock-F1
931,931,1,2016-07-28,Stock-F1
675,675,1,2016-07-28,Stock-F1
